In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Load the dataset
df = pd.read_csv("/Users/sanket/Documents/Mrunali/1st_Project/csv/train.csv")


/var/folders/jx/d374wy8j1mdfpkk2hksh8q_w0000gn/T/ipykernel_96099/1964619320.py:7: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/sanket/Documents/Mrunali/1st_Project/csv/train.csv")


In [3]:
df.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,...,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW


In [4]:
df.shape

(682154, 49)

In [5]:
# Fill missing field position
df['FieldPosition'] = df['FieldPosition'].fillna('None')

# Normalize team abbreviations
team_abbr_corrections = {'BLT': 'BAL', 'CLV': 'CLE', 'ARZ': 'ARI', 'HST': 'HOU'}
df['PossessionTeam'] = df['PossessionTeam'].replace(team_abbr_corrections)

# Normalize offense formation
df['OffenseFormation'] = df['OffenseFormation'].replace('', 'Unknown')

# Normalize game weather
def convert_weather(x):
    rain = ['Rainy', 'Rain Chance 40%', 'Showers', 'Rain', 'Light Rain']
    overcast = ['Cloudy', 'Overcast', 'Partly cloudy', 'Mostly cloudy']
    clear = ['Sunny', 'Clear', 'Fair', 'Mostly Sunny']
    snow = ['Snow', 'Heavy lake effect snow']
    indoors = ['Indoors', 'Indoor', 'Controlled Climate', 'N/A Indoor', 'N/A (Indoors)']
    if x in rain:
        return 'rain'
    elif x in overcast:
        return 'overcast'
    elif x in clear:
        return 'clear'
    elif x in snow:
        return 'snow'
    elif x in indoors:
        return 'indoors'
    else:
        return 'unknown'

df['GameWeather'] = df['GameWeather'].apply(convert_weather)


In [6]:
# Convert height to inches
df[['Height_ft', 'Height_in']] = df['PlayerHeight'].str.split('-', expand=True).astype(float)
df['PlayerHeight'] = df['Height_ft'] * 12 + df['Height_in']

# Compute player age
df['PlayerBirthDate'] = pd.to_datetime(df['PlayerBirthDate'], errors='coerce')
df['GameDate'] = pd.to_datetime(df['GameId'].astype(str).str[:8], format='%Y%m%d')
df['PlayerAge'] = (df['GameDate'] - df['PlayerBirthDate']).dt.days / 365.25

# Create isRunner flag
df['isRunner'] = df['NflId'] == df['NflIdRusher']

# Drop unnecessary columns
df.drop(['Height_ft', 'Height_in', 'PlayerBirthDate', 'Temperature', 'Humidity', 'WindSpeed',
         'WindDirection', 'StadiumType'], axis=1, inplace=True)

# Drop rows with any NA values
df.dropna(inplace=True)


In [7]:
# Standardize coordinates based on play direction
df['ToLeft'] = df['PlayDirection'] == 'left'
df['X_std'] = np.where(df['ToLeft'], 120 - df['X'], df['X']) - 10
df['Y_std'] = np.where(df['ToLeft'], (160/3) - df['Y'], df['Y'])

# Standardize direction
def standardize_direction(row):
    Dir = row['Dir']
    ToLeft = row['ToLeft']
    if ToLeft and Dir < 90:
        Dir += 360
    if not ToLeft and Dir > 270:
        Dir -= 360
    if ToLeft:
        Dir -= 180
    return Dir

df['Dir_std'] = df.apply(standardize_direction, axis=1)
df['X_std_end'] = df['S'] * np.cos(np.deg2rad(90 - df['Dir_std'])) + df['X_std']
df['Y_std_end'] = df['S'] * np.sin(np.deg2rad(90 - df['Dir_std'])) + df['Y_std']


In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Use only ball carrier data for modeling
rusher_df = df[df['isRunner']].copy()

# Select features
feature_cols = [
    'X_std', 'Y_std', 'S', 'A', 'Dis', 'Dir_std',
    'X_std_end', 'Y_std_end', 'PlayerHeight', 'PlayerWeight', 'PlayerAge',
    'Down', 'Distance', 'DefendersInTheBox'
]

# Drop any rows with missing selected features
rusher_df = rusher_df.dropna(subset=feature_cols + ['Yards'])

# Final dataset
X = rusher_df[feature_cols]
y = rusher_df['Yards']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
from xgboost import XGBRegressor

# Define and train model
xgb_model = XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = xgb_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"XGBoost MAE: {mae:.2f} yards")


XGBoost MAE: 3.75 yards


In [10]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

# Define base learners
base_learners = [
    ('ridge', Ridge(alpha=1.0)),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42))
]

# Define stacker
stacking_model = StackingRegressor(
    estimators=base_learners,
    final_estimator=XGBRegressor(n_estimators=50, learning_rate=0.05, random_state=42)
)

# Train ensemble model
stacking_model.fit(X_train, y_train)

# Evaluate
y_pred_stack = stacking_model.predict(X_test)
mae_stack = mean_absolute_error(y_test, y_pred_stack)
print(f"Stacking Ensemble MAE: {mae_stack:.2f} yards")


Stacking Ensemble MAE: 3.75 yards
